In [1]:
import laspy
import numpy as np
import pyvista as pv
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

# 1. Cargar y preprocesar datos
las = laspy.read("cloudpoint_5mm.las")
rng = np.random.default_rng(seed=42)
mask = rng.random(len(las.points)) < 0.05
subsampled = las.points[mask]

# Conversión a coordenadas reales
header = las.header
scale = header.scales
offset = header.offsets
xyz = np.vstack((subsampled.X*scale[0]+offset[0],
                 subsampled.Y*scale[1]+offset[1],
                 subsampled.Z*scale[2]+offset[2])).T

# 2. Filtrar por intensidad < 24600
intensity_mask = las.intensity[mask] < 24600  # Máscara de intensidad

# 3. Calcular min_z usando solo puntos de baja intensidad
min_z = np.min(xyz[intensity_mask, 2])  # min_z basado en puntos filtrados
lower_bound = min_z + 9

# 4. Combinar filtros de intensidad y altura
z_mask = (xyz[:, 2] > lower_bound)
combined_mask = intensity_mask & z_mask  # Puntos que cumplen ambos criterios
high_points = xyz[combined_mask]

# 5. Clustering con DBSCAN
db = DBSCAN(eps=0.5, min_samples=10).fit(high_points)
labels = db.labels_

# 6. Identificar clusters lineales usando PCA (con validación de tamaño)
cable_clusters_indices = []
for cluster_id in np.unique(labels):
    if cluster_id == -1:
        continue
    
    cluster_points = high_points[labels == cluster_id]
    
    # Validar tamaño del cluster antes de PCA
    if len(cluster_points) < 3:
        continue  # Saltar clusters con menos de 3 puntos
    
    pca = PCA(n_components=3)
    pca.fit(cluster_points)
    
    if pca.explained_variance_ratio_[0] > 0.85:
        cluster_indices = np.where(labels == cluster_id)[0]
        cable_clusters_indices.extend(cluster_indices)

# 7. Crear máscara de cables
cable_mask = np.zeros(xyz.shape[0], dtype=bool)
if cable_clusters_indices:
    high_points_indices = np.where(combined_mask)[0]
    cable_indices_in_xyz = high_points_indices[cable_clusters_indices]
    cable_mask[cable_indices_in_xyz] = True

# 8. Asignación de colores
colors = np.zeros((xyz.shape[0], 3), dtype=np.uint8)
colors[cable_mask] = [255, 0, 0]               # Rojo: cables
colors[combined_mask & ~cable_mask] = [128, 128, 128]  # Gris
colors[~combined_mask] = [128, 128, 128]       # Gris: resto

# 9. Visualización
cloud = pv.PolyData(xyz)
cloud['colors'] = colors

plotter = pv.Plotter()
plotter.add_mesh(cloud, rgb=True, style='points', point_size=2)
plotter.show()

Widget(value='<iframe src="http://localhost:39503/index.html?ui=P_0x76a42c63f860_0&reconnect=auto" class="pyvi…